# Подготовка к первому компьютерному коллоквиуму
Дан набор данных о сотрудниках. Цель - создать модель линейной регрессии, которая, основываясь на данных из файла data/train.csv, предсказывает зарплату.

В качестве метрики в задачах использовать *RMSE*, если не указано обратное.

Используйте уровень доверия 95% для всех статистических тестов, если не указано обратное.

**Задача 1.** <br>
Создайте модель линейной регрессии, используя все атрибуты. 
Выведите метрику, которую достигает модель.

In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split
from utils_nans1 import *

# загружаем данные
df = pd.read_csv('data/train.csv', sep=',')
df.head()

,plata,zvanje,oblast,godina_doktor,godina_iskustva,pol Zenski,pol Muski
0,77500,1.0,0,NaN,2,1,0
1,142253,3.0,1,49.0,60,0,1
2,124714,NaN,1,23.0,15,0,1
3,147765,3.0,1,45.0,45,0,1
4,106294,NaN,0,21.0,8,0,1


In [20]:
# Перед обучением модели необходимо решить вопросы с пропущенными значениями.
# Самый простой способ - удалить все строки.
df = df.dropna()

# обучаем модель (ищем параметры)
x = df.drop(columns=['plata'])
y = df['plata']
model = get_fitted_model(x, y)

# регистрируем метрику на тестовом наборе
df_test = pd.read_csv('data/test.csv', sep=',')
x_test = df_test.drop(columns=['plata'])
y_test = df_test['plata']
test_rmse = get_rsquared(model, x_test, y_test)
print(test_rmse)

0.7625052975730555


**Задача 2.** <br>
Когда сотрудник получает повышение (например, переходит с позиции 1 на позицию 2), какое минимальное и максимальное увеличение заработной платы он ожидает?
Объясните, являются ли найденные значения допустимыми.

**Обяснение:**  
Согласно текущей регрессионной модели, параметр, связанный с переменной "zvanje" (позив), приблизительно ~ 24480 (видно из вызова метода `summary()`). Это означает, что "plata" в среднем увеличивается на 24480$, когда "zvanje" увеличивается на 1 (работник повысился).
В задаче спрашивается о минимальном и максимальном увеличении зарплаты, когда "zvanje" увеличится на 1. Фактически нас спрашивают, каковы минимальное и максимальное значение параметра, стоящего у "zvanje". Эти два значения представляют **доверительный интервал** для переменной "zvanje". Нам нужно найти доверительный интервал для этой переменной. Чтобы найти минимальное и максимальное увеличение зарплаты, мы вызываем: `get_conf_interval(model, 'zvanje', alpha=0.05)`.
После вызова метода мы видим, что минимальное увеличение зарплаты составляет ~ 14000$, а максимальное ~ 24000$.

Однако эти значения недействительны (не являются надежными). Теория говорит, что интервалы доверия являются действительными, если выполняется предпосылка об независимости ошибок. В нашем случае предпосылка не выполняется, и, следовательно, найденные значения недействительны.


In [21]:
# ищем минимальное и максимальное значения
min_expected_raise, max_expected_raise = get_conf_interval(model, 'zvanje', alpha=0.05)
print(f'{min_expected_raise:.2f}')
print(f'{max_expected_raise:.2f}')

# проверка на то, являются ли минимальное и максимальное значения действительными
autocorrelation, _ = independence_of_errors_assumption(model, sm.add_constant(x), y, plot=False)
if autocorrelation is None:
    print('Значения действительны, потому что выполняется предпосылка о независимости ошибок.')
else:
    print('Значения недействительны.')


13999.72
24289.69
Значения недействительны.


**Задача 3.** <br>
Усовершенствовать модель так, чтобы все предположения были выполнены и значение RMSE было меньше `11 170`. Использовать разделение на *train/val* в соотношении: *80/20*. Вывести показатель, который достигает модель.

*Примечание: использовать `random state = 42` (чтобы результаты можно было воспроизвести).*

# Объяснение:
1. Нам нужно помнить о более продвинутых методах обработки пропущенных значений вместо простого удаления строк (который мы использовали в начале). Здесь мы использовали интерполяцию, потому что она дает лучшие результаты (всегда стоит пробовать разные методы).

2. Нам нужно подумать, является ли какой-то столбец избыточным (используем значения t-теста). Глядя на значения, мы можем заключить, что столбцы для пола не влияют на зарплату, поэтому мы удаляем эти столбцы => и получаем лучший показатель RMSE.
   Также важно удалить хотя бы один столбец для пола, чтобы избежать совершенной коллинеарности (чтобы удовлетворить всем предположениям, в том числе предположению о совершенной коллинеарности).

3. Данные из файла *train.csv* разделяем на обучающий и валидационный наборы, вызывая метод `train_test_split`. Важно обучать модель на обучающем наборе. Чтобы найти лучшую модель, мы смотрим на показатель качества на валидационном наборе, где цель - получить как можно меньший показатель RMSE.

4. Обязательно проверяем показатель на тестовом наборе! Это результат нашей модели. Когда мы нашли хорошие параметры модели, этот показатель будет менее 11 700 (конкретное значение - около 11 200).

5. Каждый раз, когда мы изменяем параметры модели, мы должны проверить, удовлетворяют ли все предположения (как требует задача). В этом примере удовлетворены все предположения, кроме предположения о нормальности ошибок. Мы можем игнорировать это предположение, когда у нас есть много данных (что здесь и есть, потому что у нас есть более 150 данных).


In [22]:
# Загружаем данные
df = pd.read_csv('data/train.csv', sep=',')

# Интерполируем вместо удаления значений, чтобы удовлетворить все предположения
df['zvanje'] = df['zvanje'].interpolate(method='spline', order=3, limit_direction='both')
df['godina_doktor'] = df['godina_doktor'].interpolate(method='linear', limit_direction='both')

# Удаляем колонку пола Мужчины (или пола Женщины), чтобы избежать полной коллинеарности
df = df.drop(columns=['pol Muski', 'pol Zenski'])

# Делим данные на 80-20
x = df.drop(columns=['plata'])
y = df['plata']
x_train, x_val, y_train, y_val = train_test_split(x, y, train_size=0.8, shuffle=True, random_state=42)

# Обучаем модель
model = get_fitted_model(x_train, y_train)

# Проверяем, удовлетворены ли предположения (см. объяснение)
print(are_assumptions_satisfied(model, x_train, y_train))

# Оцениваем модель на валидационном наборе данных, чтобы найти лучшую модель
val_rmse = get_rmse(model, x_val, y_val)
print(f'валидационное rmse: {val_rmse:.2f}')

# Сообщаем оценку на тестовом наборе данных
df_test = pd.read_csv('data/test.csv', sep=',')
x_test = df_test.drop(columns=['plata', 'pol Muski', 'pol Zenski'])
y_test = df_test['plata']
test_rmse = get_rmse(model, x_test, y_test)
print(f'тестовое rmse: {test_rmse:.2f}')

False
валидационное rmse: 21029.75
тестовое rmse: 11165.79


**Задача 4.** <br>
Существует ли хотя бы одна независимая переменная, линейно связанная с зарплатой? Обоснуйте ваш ответ.

In [23]:
# Вызовом model.summary() видим, что p-значение F-теста составляет 2.44e-30, что меньше 5%.
# Поэтому мы с уверенностью 95% можем утверждать, что по крайней мере одна независимая переменная имеет линейную связь с зарплатой.
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  plata   R-squared:                       0.463
Model:                            OLS   Adj. R-squared:                  0.454
Method:                 Least Squares   F-statistic:                     50.04
Date:                Wed, 06 Dec 2023   Prob (F-statistic):           2.52e-30
Time:                        06:19:24   Log-Likelihood:                -2688.8
No. Observations:                 237   AIC:                             5388.
Df Residuals:                     232   BIC:                             5405.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const            4.407e+04   5502.727     

**Задача 5.** <br>
Объясните меру скорректированного $r^2$.

In [24]:
# Этот вопрос чисто теоретический. 
# Ответ на него можно найти в материалах для лекций и практических занятий, когда рассматривалась множественная регрессия.